In [1]:
import pickle
import numpy as np

# ============================================================
# CONFIG
# ============================================================
P_TORT = "/home/admin/Ana/MicroBrain/output/graph_18_OutGeom.pkl"
P_NON  = "/home/admin/Ana/MicroBrain/18_igraph.pkl"

N_SAMPLE = 2000          # pon None para todos
TOL_ALIGN = 1e-12        # para checks numéricos

# ============================================================
# Helpers
# ============================================================
def load_pkl(path):
    with open(path, "rb") as f:
        return pickle.load(f)

def pick_first_existing_attr(attr_list, candidates):
    for c in candidates:
        if c in attr_list:
            return c
    return None

def weighted_mean(vals, weights):
    v = np.asarray(vals, float)
    w = np.asarray(weights, float)
    sw = np.sum(w)
    return float(np.sum(v * w) / sw) if sw > 0 else np.nan

def safe_sample_indices(n_total, n_sample):
    if n_sample is None or n_sample >= n_total:
        return np.arange(n_total, dtype=int)
    rng = np.random.default_rng(0)
    return rng.choice(n_total, size=int(n_sample), replace=False)

def edgelist_array(G):
    return np.asarray(G.get_edgelist(), dtype=np.int64)  # (nE,2)

def sort_edge_uv(E):
    """Convierte (u,v) en (min,max) para comparar undirected."""
    u = np.minimum(E[:,0], E[:,1])
    v = np.maximum(E[:,0], E[:,1])
    return np.column_stack([u,v])

# ============================================================
# Load
# ============================================================
data_tort = load_pkl(P_TORT)
G_tort = data_tort["graph"]

G_non = load_pkl(P_NON)
# si tu NON también es pseudo-json en vez de igraph directo, descomenta:
# if isinstance(G_non, dict) and "graph" in G_non:
#     G_non = G_non["graph"]

print("TORT:", G_tort.summary())
print("NON :", G_non.summary())

# ============================================================
# Detect attrs in NON
# ============================================================
edge_attrs_non = G_non.es.attributes()
print("\nNON edge attrs:", edge_attrs_non)

radius_candidates = ["radius", "r_edge", "radii_edge", "rad", "r", "radius_um", "r_um", "diameter", "diam", "diam_um"]
length_candidates = ["length", "len", "length_um", "L", "edge_length", "length_geom", "length_eucl"]

rad_attr = pick_first_existing_attr(edge_attrs_non, radius_candidates)
len_attr = pick_first_existing_attr(edge_attrs_non, length_candidates)

if rad_attr is None:
    raise KeyError("No encuentro atributo de radio/diámetro en NON edges.")
if len_attr is None:
    raise KeyError("No encuentro atributo de length en NON edges.")

print("Picked NON radius attr:", rad_attr)
print("Picked NON length attr:", len_attr)

# NON radius array
r_non_raw = np.asarray(G_non.es[rad_attr], dtype=float)
if rad_attr in ["diameter", "diam", "diam_um"]:
    r_edge_non = 0.5 * r_non_raw
else:
    r_edge_non = r_non_raw

L_edge_non = np.asarray(G_non.es[len_attr], dtype=float)

# ============================================================
# TORT per-point radii + geom_start/end
# ============================================================
geom = data_tort.get("geom", {})
if "radii" not in geom:
    raise KeyError("En TORT full espero data['geom']['radii'] (radio por punto) y no está.")
r_point = np.asarray(geom["radii"], dtype=float)

# geom_start/end deberían estar en G_tort
if "geom_start" not in G_tort.es.attributes() or "geom_end" not in G_tort.es.attributes():
    raise KeyError("En TORT G.es faltan geom_start / geom_end.")

gs = np.asarray(G_tort.es["geom_start"], dtype=np.int64)
ge = np.asarray(G_tort.es["geom_end"], dtype=np.int64)

# ============================================================
# 0) Check alignment NON vs TORT (edge ordering)
# ============================================================
E_non  = edgelist_array(G_non)
E_tort = edgelist_array(G_tort)

nE = min(E_non.shape[0], E_tort.shape[0])
if E_non.shape[0] != E_tort.shape[0]:
    print(f"\n:warning: ecount distinto: NON={E_non.shape[0]} TORT={E_tort.shape[0]}. Usaré min={nE}.")
else:
    print(f"\n:white_check_mark: ecount igual: {nE}")

# compare undirected pairs
En = sort_edge_uv(E_non[:nE])
Et = sort_edge_uv(E_tort[:nE])

same_order = np.all(En == Et)
if same_order:
    print(":white_check_mark: Edge ordering coincide (por endpoints). Puedes comparar por index directo.")
    map_tort_idx_for_non = np.arange(nE, dtype=int)  # identidad
else:
    print(":warning: Edge ordering NO coincide. Haré mapping por endpoints (u,v).")

    # mapping endpoints -> list de idx en tort (por si hay multiedges)
    tort_dict = {}
    for i in range(nE):
        key = (int(Et[i,0]), int(Et[i,1]))
        tort_dict.setdefault(key, []).append(i)

    # map non idx -> tort idx (primero disponible)
    map_tort_idx_for_non = np.full(nE, -1, dtype=int)
    used = set()
    for i in range(nE):
        key = (int(En[i,0]), int(En[i,1]))
        cand = tort_dict.get(key, [])
        # si hay multiedges, intenta asignar uno no usado
        pick = -1
        for j in cand:
            if j not in used:
                pick = j
                used.add(j)
                break
        map_tort_idx_for_non[i] = pick

    ok = np.sum(map_tort_idx_for_non >= 0)
    print(f"Mapping por endpoints: {ok}/{nE} edges asignados.")
    if ok < nE * 0.999:
        print(":x: Demasiados edges sin mapear. Esto sugiere multiedges fuertes o graphs no equivalentes.")
        print("   En ese caso lo más robusto es guardar un orig_eid en el pipeline.")
        # seguimos pero con los que sí están mapeados

# ============================================================
# 1) Compute per-edge comparisons
# ============================================================
idx_all = np.arange(nE, dtype=int)
idx_all = idx_all[map_tort_idx_for_non >= 0]  # solo mapeados
idx = safe_sample_indices(len(idx_all), N_SAMPLE)
idx_non = idx_all[idx]

diff_r_max = []
diff_r_mean = []

D_non = []
D_tort_mean = []
D_tort_max = []
W = []   # weights

bad = []

for k in idx_non:
    i_non = int(k)
    i_tort = int(map_tort_idx_for_non[i_non])

    s = int(gs[i_tort])
    en = int(ge[i_tort])
    if en - s < 2:
        bad.append(i_non);
        continue
    rp = r_point[s:en]
    if rp.size == 0:
        bad.append(i_non);
        continue

    r_e = float(r_edge_non[i_non])
    rmax = float(np.nanmax(rp))
    rmean = float(np.nanmean(rp))

    diff_r_max.append(r_e - rmax)
    diff_r_mean.append(r_e - rmean)

    D_non.append(2.0 * r_e)
    D_tort_mean.append(2.0 * rmean)
    D_tort_max.append(2.0 * rmax)

    w = float(L_edge_non[i_non])
    W.append(w)

diff_r_max = np.asarray(diff_r_max, float)
diff_r_mean = np.asarray(diff_r_mean, float)

D_non = np.asarray(D_non, float)
D_tort_mean = np.asarray(D_tort_mean, float)
D_tort_max = np.asarray(D_tort_max, float)
W = np.asarray(W, float)

# ============================================================
# 2) Summary
# ============================================================
print("\n==============================")
print(" SANITY CHECK FULL (NON vs TORT) ")
print("==============================\n")

print(f"Edges compared: {len(D_non)}  (bad/skipped: {len(bad)})")
print("\n1) ¿r_edge NON == max(r_pts TORT)?")
print(f"mean(r_non - rmax):   {np.mean(diff_r_max):.6e}")
print(f"std :                 {np.std(diff_r_max):.6e}")
print(f"maxabs:               {np.max(np.abs(diff_r_max)):.6e}")

print("\n2) Sesgo por usar MAX vs MEAN")
print(f"mean(r_non - rmean):  {np.mean(diff_r_mean):.6e}")

D_non_w  = weighted_mean(D_non, W)
D_mean_w = weighted_mean(D_tort_mean, W)
D_max_w  = weighted_mean(D_tort_max, W)

print("\n3) Diámetros ponderados por length NON")
print(f"DIÁMETRO NON (edge radius):     {D_non_w:.6f} µm")
print(f"DIÁMETRO TORT (mean per-point): {D_mean_w:.6f} µm")
print(f"DIÁMETRO TORT (max  per-point): {D_max_w:.6f} µm")

bias_pct = ((D_non_w - D_mean_w) / D_mean_w) * 100 if np.isfinite(D_mean_w) and D_mean_w != 0 else np.nan
print(f"\n=> Sesgo NON vs TORT(mean): {bias_pct:.2f}%")

# ejemplos extremos
if len(diff_r_max) > 0:
    j = int(np.argmax(np.abs(diff_r_max)))
    print("\nEjemplo edge con mayor |r_non-rmax|:")
    print(f"  r_non={0.5*D_non[j]:.6f}  rmax={0.5*D_tort_max[j]:.6f}  diff={diff_r_max[j]:.6e}")

print("\nDONE.")

TORT: IGRAPH U--- 3136928 4620894 -- 
+ attr: id (v), diameter (e), geom_end (e), geom_start (e), length (e), length_tortuous (e), nkind (e), radius (e), tortuosity (e)
NON : IGRAPH U--- 3136928 4620894 -- 
+ attr: annotation (v), coords (v), distance_to_surface (v), id (v), radii (v), connectivity (e), diameter (e), length (e), nkind (e), radius (e)

NON edge attrs: ['connectivity', 'nkind', 'radius', 'diameter', 'length']
Picked NON radius attr: radius
Picked NON length attr: length

:white_check_mark: ecount igual: 4620894
:white_check_mark: Edge ordering coincide (por endpoints). Puedes comparar por index directo.

 SANITY CHECK FULL (NON vs TORT) 

Edges compared: 2000  (bad/skipped: 0)

1) ¿r_edge NON == max(r_pts TORT)?
mean(r_non - rmax):   -9.646497e-09
std :                 8.942317e-08
maxabs:               8.643254e-07

2) Sesgo por usar MAX vs MEAN
mean(r_non - rmean):  7.606618e-01

3) Diámetros ponderados por length NON
DIÁMETRO NON (edge radius):     7.364773 µm
DIÁMETR